__Note from the student:__  
*Lines marked '##' are my own notes. These may include resources I've used, but they're primarily for  pseudo-code and my own thought process so that I can "talk myself through it" when reviewing the code later.*  

**Note to self:**  
*You got this!*

# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk
...,...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,own,86600,0.653580,12,2,56600,high_risk
77532,17700.0,10.662,mortgage,80900,0.629172,11,2,50900,high_risk
77533,17600.0,10.595,rent,80300,0.626401,11,2,50300,high_risk
77534,16300.0,10.068,mortgage,75300,0.601594,10,2,45300,high_risk


In [4]:
## Checked for null values
print(df.isna().count())
df.describe()

loan_size           77536
interest_rate       77536
homeowner           77536
borrower_income     77536
debt_to_income      77536
num_of_accounts     77536
derogatory_marks    77536
total_debt          77536
loan_status         77536
dtype: int64


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [5]:
## Replaced the string values in the 'homeowner' and 'loan_status' columns with numeric values.
## Dropped num_of_accounts column as it seemed to be irrelevant
lending_df = df.drop(columns=['num_of_accounts']).replace(
    {'homeowner': {
        'own': 0,
        'mortgage': 1,
        'rent': 2
        },
     'loan_status': {
         'low_risk': 0,
         'high_risk': 1
         }
    }
)
lending_df

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,0,52800,0.431818,1,22800,0
1,8400.0,6.692,0,43600,0.311927,0,13600,0
2,9000.0,6.963,2,46100,0.349241,0,16100,0
3,10700.0,7.664,0,52700,0.430740,1,22700,0
4,10800.0,7.698,1,53000,0.433962,1,23000,0
...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,0,86600,0.653580,2,56600,1
77532,17700.0,10.662,1,80900,0.629172,2,50900,1
77533,17600.0,10.595,2,80300,0.626401,2,50300,1
77534,16300.0,10.068,1,75300,0.601594,2,45300,1


# Split the Data into Training and Testing

In [6]:
# Create our features
## \10-Time-Series\3\03-Ins_Time_Series_Linear_Regression\Solved\time_series_simple_regression.ipynb
## \10-Time-Series\3\05-Ins_Train_Test\Solved\train_test.ipynb

X = lending_df
X.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,0,52800,0.431818,1,22800,0
1,8400.0,6.692,0,43600,0.311927,0,13600,0
2,9000.0,6.963,2,46100,0.349241,0,16100,0
3,10700.0,7.664,0,52700,0.430740,1,22700,0
4,10800.0,7.698,1,53000,0.433962,1,23000,0


In [7]:
# Create our target
## Assigned y to the 'loan_status' column
y = lending_df['loan_status']
y

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [8]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,derogatory_marks,total_debt,loan_status
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.704705,49221.949804,0.377318,0.392308,19221.949804,0.032243
std,2093.223153,0.889495,0.644464,8371.635077,0.081519,0.582086,8371.635077,0.176646
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,0.000000,14800.000000,0.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,0.000000,18100.000000,0.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,1.000000,21400.000000,0.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,3.000000,75200.000000,1.000000


In [9]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

In [10]:
# Create X_train, X_test, y_train, y_test
## \11-Classification\1\01-Ins_Logistic_Regression\Solved\logistic_regression.ipynb
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    )
X_train.shape

(58152, 8)

In [11]:
X_train.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,derogatory_marks,total_debt,loan_status
29175,8600.0,6.792,0,44500,0.325843,0,14500,0
23020,7800.0,6.419,0,41000,0.268293,0,11000,0
31269,10000.0,7.386,0,50100,0.401198,1,20100,0
35479,9300.0,7.093,2,47300,0.365751,0,17300,0
13470,9200.0,7.045,0,46900,0.360341,0,16900,0


In [12]:
y_train.head()

29175    0
23020    0
31269    0
35479    0
13470    0
Name: loan_status, dtype: int64

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

*From [scikit learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html):*

"Standardize features by removing the mean and scaling to unit variance.  
The standard score of a sample x is calculated as:

z = (x - u) / s

where **u** is the mean of the training samples or zero if with_mean=False,  
and **s** is the standard deviation of the training samples or one if with_std=False."

In [13]:
# Create the StandardScaler instance
## \11-Classification\2\01-Ins-Categorical-Data\Solved\categorical-data.ipynb
## \11-Classification\2\03-Ins_Decision-Trees\Solved\decision-trees.ipynb
## \11-Classification\2\05-Ins_Random_Forest\Solved\random-forest.ipynb
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [14]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler.fit(X_train)

print(scaler.mean_)
print(scaler.scale_)

[9.80798253e+03 7.29337589e+00 7.05186408e-01 4.92317530e+04
 3.77464521e-01 3.92075939e-01 1.92317530e+04 3.23462650e-02]
[2.09323248e+03 8.89474288e-01 6.43866313e-01 8.37135557e+03
 8.13841899e-02 5.82667238e-01 8.37135557e+03 1.76918015e-01]


In [15]:
# Scale the training and testing data
scaled_X_train = scaler.transform(X_train)
scaled_X_train[:5]

array([[-0.57708952, -0.56367666, -1.09523731, -0.5652314 , -0.63429794,
        -0.67289855, -0.5652314 , -0.18283195],
       [-0.95927354, -0.98302549, -1.09523731, -0.98332378, -1.34143791,
        -0.67289855, -0.98332378, -0.18283195],
       [ 0.09173251,  0.10413354, -1.09523731,  0.10371642,  0.29161787,
         1.04334691,  0.10371642, -0.18283195],
       [-0.24267851, -0.22527452,  2.01099757, -0.23075749, -0.14393449,
        -0.67289855, -0.23075749, -0.18283195],
       [-0.29045151, -0.27923898, -1.09523731, -0.27853948, -0.21040167,
        -0.67289855, -0.27853948, -0.18283195]])

In [16]:
scaled_X_test = scaler.transform(X_test)
scaled_X_test[:5]

array([[ 1.33383057,  1.32170668, -1.09523731,  1.32215708,  1.53620797,
         1.04334691,  1.32215708, -0.18283195],
       [-0.00381349, -0.00491964,  2.01099757, -0.00379305,  0.15702536,
        -0.67289855, -0.00379305, -0.18283195],
       [ 0.52168953,  0.53584922,  0.45788013,  0.5337543 ,  0.7848731 ,
         1.04334691,  0.5337543 , -0.18283195],
       [ 0.42614353,  0.41892623,  0.45788013,  0.41429933,  0.65461753,
         1.04334691,  0.41429933, -0.18283195],
       [ 0.04395951,  0.06703297,  0.45788013,  0.06787993,  0.24729423,
        -0.67289855,  0.06787993, -0.18283195]])

# Simple Logistic Regression

In [17]:
## \11-Classification\1\01-Ins_Logistic_Regression\Solved\logistic_regression.ipynb
from sklearn.linear_model import LogisticRegression
logreg_model = LogisticRegression(solver='lbfgs', random_state=1)
logreg_model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
logreg_y_pred = logreg_model.predict(X_test)
balanced_accuracy_score(y_test, logreg_y_pred)

1.0

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, logreg_y_pred)

array([[18765,     0],
       [    0,   619]], dtype=int64)

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, logreg_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00     18765
          1       1.00      1.00      1.00      1.00      1.00      1.00       619

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [21]:
# Resample the training data with the RandomOversampler
## 11-Classification\3\04-Stu_Do_More_Loans\Solved\more_loans.ipynb
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
ros_X_resampled, ros_y_resampled = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(ros_y_resampled)

Counter({0: 56271, 1: 56271})

In [22]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

ros_model = LogisticRegression(solver='lbfgs', random_state=1)
ros_model.fit(ros_X_resampled, ros_y_resampled)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
ros_y_pred = ros_model.predict(X_test)
balanced_accuracy_score(y_test, ros_y_pred)

0.9936781215845847

In [24]:
# Display the confusion matrix
confusion_matrix(y_test, ros_y_pred)

array([[18649,   116],
       [    4,   615]], dtype=int64)

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, ros_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.99      1.00      0.99      0.99     18765
          1       0.84      0.99      0.99      0.91      0.99      0.99       619

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [26]:
# Resample the training data with SMOTE
## \11-Classification\3\04-Stu_Do_More_Loans\Solved\more_loans.ipynb
from imblearn.over_sampling import SMOTE

smote_X_resampled, smote_y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(smote_y_resampled)

Counter({0: 56271, 1: 56271})

In [27]:
# Train the Logistic Regression model using the resampled data
smote_model = LogisticRegression(solver='lbfgs', random_state=1)
smote_model.fit(smote_X_resampled, smote_y_resampled)

LogisticRegression(random_state=1)

In [28]:
# Calculated the balanced accuracy score
smote_y_pred = smote_model.predict(X_test)
balanced_accuracy_score(y_test, smote_y_pred)

0.9979581655085195

In [29]:
# Display the confusion matrix
confusion_matrix(y_test, smote_y_pred)

array([[18749,    16],
       [    2,   617]], dtype=int64)

In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, smote_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00     18765
          1       0.97      1.00      1.00      0.99      1.00      1.00       619

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the data using the ClusterCentroids resampler
## \11-Classification\3\05-Stu_Do_Undersampling\Solved\undersampling.ipynb
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
cc_X_resampled, cc_y_resampled = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(cc_y_resampled)

Counter({0: 1881, 1: 1881})

In [32]:
# Train the Logistic Regression model using the resampled data
cc_model = LogisticRegression(solver='lbfgs', random_state=1)
cc_model.fit(cc_X_resampled, cc_y_resampled)

LogisticRegression(random_state=1)

In [33]:
# Calculate the balanced accuracy score
cc_y_pred = cc_model.predict(X_test)
balanced_accuracy_score(y_test, cc_y_pred)

0.9865149130022852

In [34]:
# Display the confusion matrix
confusion_matrix(y_test, cc_y_pred)

array([[18653,   112],
       [   13,   606]], dtype=int64)

In [35]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, cc_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.98      1.00      0.99      0.97     18765
          1       0.84      0.98      0.99      0.91      0.99      0.97       619

avg / total       0.99      0.99      0.98      0.99      0.99      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [36]:
# Resample the training data with SMOTEENN
## \11-Classification\3\06-Ins_Do_Combination_Sampling\Solved\combination_sampling.ipynb
from imblearn.combine import SMOTEENN

eenn = SMOTEENN(random_state=1)
eenn_X_resampled, eenn_y_resampled = eenn.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(eenn_y_resampled)

Counter({0: 55954, 1: 56073})

In [37]:
# Train the Logistic Regression model using the resampled data
eenn_model = LogisticRegression(solver='lbfgs', random_state=1)
eenn_model.fit(eenn_X_resampled, eenn_y_resampled)

LogisticRegression(random_state=1)

In [38]:
# Calculate the balanced accuracy score
eenn_y_pred = eenn_model.predict(X_test)
balanced_accuracy_score(y_test, eenn_y_pred)

0.9985611510791367

In [39]:
# Display the confusion matrix
confusion_matrix(y_test, eenn_y_pred)

array([[18711,    54],
       [    0,   619]], dtype=int64)

In [40]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, eenn_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00     18765
          1       0.92      1.00      1.00      0.96      1.00      1.00       619

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     19384



# Final Questions

1. Which model had the best balanced accuracy score?  
   SMOTE oversampling with a score of 0.9979581655085195  
<br>
2. Which model had the best recall score?  
    SMOTE oversampling with a score of 18,749  
<br>
3. Which model had the best geometric mean score?  
   SMOTE oversampling and SMOTEENN combination sampling both had a mean score of 1.00  
